# ML3_LSTM_MNIST - Tania_El_Achkar

In [1]:
import time
start = time.time()

In [2]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
Long Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf
'''

'\nA Recurrent Neural Network (LSTM) implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)\nLong Short Term Memory paper: http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf\n'

In [3]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
'''
To classify images using a recurrent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

'\nTo classify images using a recurrent neural network, we consider every image\nrow as a sequence of pixels. Because MNIST image shape is 28*28px, we will then\nhandle 28 sequences of 28 steps for every sample.\n'

In [6]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

In [7]:
# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features 
n_classes = 10 # MNIST total classes (0-9 digits)

In [8]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

In [9]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [10]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    print("x size before unstack:", x.get_shape())
    x = tf.unstack(x, n_steps, 1)
    print(x)
    print("x size after unstack:", len(x))
    exit(-1)
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

x size before unstack: (?, 28, 28)
[<tf.Tensor 'unstack:0' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:4' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:5' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:6' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:7' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:8' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:9' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:10' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:11' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:12' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:13' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:14' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:15' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:16' shape=(?, 28) dtype=float32>, <tf.Tensor 'unstack:17' shape=(?, 28) dtype=float32>, <t

In [11]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
# Initializing the variables
init = tf.global_variables_initializer()

In [14]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step * batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.930382, Training Accuracy= 0.31250
Iter 2560, Minibatch Loss= 1.656423, Training Accuracy= 0.43750
Iter 3840, Minibatch Loss= 1.255198, Training Accuracy= 0.55469
Iter 5120, Minibatch Loss= 1.138133, Training Accuracy= 0.54688
Iter 6400, Minibatch Loss= 1.050768, Training Accuracy= 0.70312
Iter 7680, Minibatch Loss= 0.893175, Training Accuracy= 0.68750
Iter 8960, Minibatch Loss= 0.750221, Training Accuracy= 0.78125
Iter 10240, Minibatch Loss= 0.665614, Training Accuracy= 0.78125
Iter 11520, Minibatch Loss= 0.435369, Training Accuracy= 0.88281
Iter 12800, Minibatch Loss= 0.408342, Training Accuracy= 0.89062
Iter 14080, Minibatch Loss= 0.534957, Training Accuracy= 0.80469
Iter 15360, Minibatch Loss= 0.460466, Training Accuracy= 0.89062
Iter 16640, Minibatch Loss= 0.395592, Training Accuracy= 0.86719
Iter 17920, Minibatch Loss= 0.516093, Training Accuracy= 0.79688
Iter 19200, Minibatch Loss= 0.274114, Training Accuracy= 0.92969
Iter 20480, Minibatch Loss= 0.30

In [15]:
end = time.time()
print("Time: " + str(end - start))

Time: 72.9856481552124
